In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:90% !important; }</style>"))
import numpy as np
import pandas as pd
import h5py
from tqdm import tqdm
import scipy.io as sio
from scipy import stats
from importlib import reload
from matplotlib import pyplot as plt
import sys
#sys.path.append('/home/jma819/post_cmfe_analysis')
sys.path.append('/Users/johnmarshall/Documents/Analysis/PythonAnalysisScripts/post_cmfe_analysis')
import python_utils_jjm as utils_jjm
import dlc_utils
import caiman
import statsmodels.formula.api as smf
import scipy.spatial.distance as dist
import itertools
import math
import warnings
import numbers
from multiprocessing import Pool
from functools import partial
warnings.filterwarnings(action='once')

C:\Users\scanimage\AppData\Local\Temp\ipykernel_1824\3734102346.py:1: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display, HTML


ModuleNotFoundError: No module named 'python_utils_jjm'

In [2]:
#import matlab.engine 
#import os

In [3]:
#eng = matlab.engine.start_matlab()

In [4]:
#os.chdir('/Users/johnmarshall/Documents/Analysis/PythonAnalysisScripts/post_cmfe_analysis/openfield_analysis/spatial_clustering/jones_scripts')
#eng.cd('/Users/johnmarshall/Documents/Analysis/PythonAnalysisScripts/post_cmfe_analysis/openfield_analysis/spatial_clustering/jones_scripts')

In [5]:
## import Jones's functions
#eng.help(eng.getPaddedSignalPeaks())

In [6]:
reload(utils_jjm)
reload(dlc_utils)

<module 'dlc_utils' from '/Users/johnmarshall/Documents/Analysis/PythonAnalysisScripts/post_cmfe_analysis/dlc_utils.py'>

In [7]:
cnmfe_file_key = pd.read_csv('/Volumes/My_Passport/cnmfe_analysis_files/OpenFieldAnalysis/2020/cnmfe_file_key.csv')
#cnmfe_file_key = pd.read_csv('/projects/p30771/dlc_analysis/openfield_dlc_output/cnmfe_file_key.csv')

In [8]:
cnmfe_base_dir = '/Volumes/My_Passport/cnmfe_analysis_files/batch_output_files/'
#cnmfe_base_dir = '/projects/p30771/MATLAB/CNMF_E_jjm/quest_MATLAB_cnmfe/batch_output_files/'
wt_CNMFE_file = ['30-Mar_20_39_05_out.mat', '30-Mar_20_45_16_out.mat', '27-Feb_17_33_59_out.mat', '22-Mar_22_52_02_out.mat',
                 '28-Feb_16_10_05_out.mat', '27-Feb_17_32_15_out.mat', '28-Feb_16_21_21_out.mat', '25-Mar_13_27_27_out.mat',
                 '25-Mar_14_22_02_out.mat', '25-Mar_14_22_44_out.mat', '26-Mar_18_33_55_out.mat', '27-Mar_00_26_12_out.mat', '27-Mar_00_48_46_out.mat']

ko_CNMFE_files = ['31-Mar_13_28_15_out.mat', '29-Mar_21_42_20_out.mat', '13-Apr_17_57_40_out.mat', '29-Mar_14_27_55_out.mat', '13-Apr_16_01_20_out.mat',
                 '13-Apr_16_11_27_out.mat', '29-Mar_13_39_44_out.mat']

In [9]:
tracking_files = ['/Volumes/My_Passport/dlc_analysis/behavcamvideos/open_field_miniscope/'+utils_jjm.find_behavior_tracking(fname, cnmfe_file_key) for fname in ko_CNMFE_files] 
#tracking_files = ['/projects/p30771/dlc_analysis/openfield_dlc_output/'+utils_jjm.find_behavior_tracking(fname, cnmfe_file_key) for fname in wt_CNMFE_file] 

In [10]:
CNMFE_dir_paths_list = [str(cnmfe_base_dir+fname) for fname in ko_CNMFE_files]
#CNMFE_dir_paths_list

In [11]:
binning_time = 1 # seconds 
body_part_for_tracking = 'tail_base' #
number_of_bins = 50 #
polynomial_degree = 2 #

grouped_raw_data = {}
success = []
failed = []
for CNMFE_file, tracking_file in zip(CNMFE_dir_paths_list, tracking_files):
    #print(CNMFE_file)
    #print(tracking_file)
    try:
        # load cell fluorescence 
        cell_fluorescence = sio.loadmat(CNMFE_file)
        C_timedelta = utils_jjm.create_fluorescence_time_delta(cell_fluorescence['C'])
        C_normalized = C_timedelta.apply(utils_jjm.normalize).set_index(pd.to_timedelta(np.linspace(0, (len(C_timedelta)-1)*(1/20), len(C_timedelta)), unit='s'), drop=True)
        C_z_scored = C_timedelta.apply(stats.zscore).set_index(pd.to_timedelta(np.linspace(0, (len(C_timedelta)-1)*(1/20), len(C_timedelta)), unit='s'), drop=True)
        C_normalized_z_scored = C_normalized.apply(stats.zscore).set_index(pd.to_timedelta(np.linspace(0, (len(C_normalized)-1)*(1/20), len(C_normalized)), unit='s'), drop=True)
        # create tracking time deltas
        interpolated = utils_jjm.prepare_timedelta_dfs(tracking_file)
        #load spatial components by session
        com_df, spatial_components = utils_jjm.return_spatial_info(CNMFE_file, 0.6)
        cell_contours, for_dims = utils_jjm.create_contour_layouts(spatial_components)
        #C_z_scored_filtered = utils_jjm.filter_out_by_size(C_z_scored, cell_contours, for_dims, 0.6, 100)
        #store results 
        grouped_raw_data[tracking_file.split('/')[-2]] = {'C': C_timedelta, 'C_z_scored': C_z_scored, 'C_normalized': C_normalized, 'C_normalized_z_scored': C_normalized_z_scored, 
                                                          'interpolated' : interpolated, 'com' : com_df, 'spatial_components' : spatial_components, 'cell_contours': cell_contours,  
                                                         'for_dims' : for_dims}
        success.append((tracking_file.split('/')[-2], CNMFE_file.split('/')[-1]))
    except FileNotFoundError:
        failed.append(tracking_file)
    except OSError:
        failed.append(tracking_file)

## spatial clustering

In [12]:
## combine dfs for each session to bin velocity values across dfs
#downsample
new_sampling_interval = .2
V_df = pd.concat([grouped_raw_data[session]['interpolated'].resample(str(new_sampling_interval)+'S').max() 
                  for session in list(grouped_raw_data.keys())], keys=list(grouped_raw_data.keys()))
all_sessions_v_bins = pd.cut(V_df['tail_base'], bins=50)

V_df['velocity_bins'] = all_sessions_v_bins

In [26]:
V_df.loc['GRIN012_H17_M32_S17']

index  centroid  ear_right  miniscope_light  ear_left  \
00:00:00          13.500000  2.515226   3.103374         3.988434  2.562785   
00:00:00.200000   17.500000  2.460201   3.317561         3.809521  2.468187   
00:00:00.400000   21.500000  2.405176   3.531748         3.630609  2.373588   
00:00:00.600000   25.500000  2.350151   3.745935         3.451696  2.278990   
00:00:00.800000   29.500000  2.295127   3.960121         3.272783  2.184391   
...                     ...       ...        ...              ...       ...   
00:17:20.200000  828.745625  6.051251   6.068077        11.766592  7.764174   
00:17:20.400000  832.343125  6.399855   4.627216        15.371330  5.978581   
00:17:20.600000  835.940625  6.748460   3.186355        18.976068  4.192988   
00:17:20.800000  839.538125  7.097064   1.745493        22.580805  2.407395   
00:17:21         840.437500  7.184215   0.304632        23.481990  0.621802   

                 tail_base  msCam_index  centroid_likelihood  \
00:00:00          2.890625          3.0                  NaN   
00:00:00.200000   2.692109          7.0                  NaN   
00:00:00.400000   2.493592         11.0                  NaN   
00:00:00.600000   2.295075         15.0                  NaN   
00:00:00.800000   2.096559         19.0                  NaN   
...                    ...          ...                  ...   
00:17:20.200000   6.352312      20807.0                  NaN   
00:17:20.400000   6.454261      20811.0                  NaN   
00:17:20.600000   6.556209      20815.0                  NaN   
00:17:20.800000   6.658158      20819.0                  NaN   
00:17:21          6.683645      20820.0                  NaN   

                 ear_right_likelihood  miniscope_light_likelihood  \
00:00:00                          NaN                         NaN   
00:00:00.200000                   NaN                         NaN   
00:00:00.400000                   NaN                         NaN   
00:00:00.600000                   NaN                         NaN   
00:00:00.800000                   NaN                         NaN   
...                               ...                         ...   
00:17:20.200000                   NaN                         NaN   
00:17:20.400000                   NaN                         NaN   
00:17:20.600000                   NaN                         NaN   
00:17:20.800000                   NaN                         NaN   
00:17:21                          NaN                         NaN   

                 ear_left_likelihood  tail_base_likelihood     velocity_bins  
00:00:00                         NaN                   NaN     (2.829, 5.58]  
00:00:00.200000                  NaN                   NaN  (-0.0606, 2.829]  
00:00:00.400000                  NaN                   NaN  (-0.0606, 2.829]  
00:00:00.600000                  NaN                   NaN  (-0.0606, 2.829]  
00:00:00.800000                  NaN                   NaN  (-0.0606, 2.829]  
...                              ...                   ...               ...  
00:17:20.200000                  NaN                   NaN     (5.58, 8.332]  
00:17:20.400000                  NaN                   NaN     (5.58, 8.332]  
00:17:20.600000                  NaN                   NaN     (5.58, 8.332]  
00:17:20.800000                  NaN                   NaN     (5.58, 8.332]  
00:17:21                         NaN                   NaN     (5.58, 8.332]  

[5206 rows x 13 columns]

In [27]:
# compile and filter fluorescence
filtered_for_analysis = {}
for session in list(grouped_raw_data.keys()):
    filtered_for_analysis[session] = utils_jjm.filter_out_by_size(grouped_raw_data[session]['C_normalized_z_scored'], grouped_raw_data[session]['cell_contours'], 
                                                                  grouped_raw_data[session]['for_dims'], 0.6, 100)
C_df = pd.concat([filtered_for_analysis[session].resample(str(new_sampling_interval)+'S').max()
                  for session in list(grouped_raw_data.keys())], keys=list(grouped_raw_data.keys()))
C_df['velocity_bins'] = all_sessions_v_bins
#if want to normalize bins by each indiv mouse 
#C_df['by_mouse_vbins'] = V_df['by_mouse_vbins']

In [32]:
C_df.loc['GRIN012_H17_M32_S17'][C_df.loc['GRIN012_H17_M32_S17']['velocity_bins'].notna()].iloc[:,:].dropna(axis=1)

1         2         3         4         7         8  \
00:00:00        -0.376794 -0.004745  0.450959 -0.309227 -0.221392 -0.278460   
00:00:00.200000 -0.306167  0.102310  0.345854 -0.273399 -0.190540 -0.038487   
00:00:00.400000 -0.316236  0.000900  0.253256 -0.282653 -0.220512 -0.068930   
00:00:00.600000 -0.328850 -0.113600  1.501771 -0.316145 -0.261911 -0.177432   
00:00:00.800000 -0.340596 -0.208212  2.650304 -0.344636 -0.358440 -0.267445   
...                   ...       ...       ...       ...       ...       ...   
00:17:20.200000 -0.324722 -0.355914  0.930279 -0.492554 -0.751163 -0.961469   
00:17:20.400000  0.194894  0.040039  1.059677 -0.494145 -0.751163 -0.103260   
00:17:20.600000 -0.192350 -0.353085  0.798822 -0.495559 -0.751163 -0.535972   
00:17:20.800000 -0.205112 -0.089171  1.040609 -0.496817 -0.751163 -0.721540   
00:17:21         0.220509 -0.274594  0.934513 -0.497935 -0.751163 -0.719434   

                       10        11        12        13  ...        58  \
00:00:00        -0.345085 -0.439040 -0.226133  0.154454  ... -0.662661   
00:00:00.200000 -0.363913 -0.470494 -0.242395  0.046148  ... -0.262511   
00:00:00.400000 -0.431485 -0.130533 -0.271917 -0.041017  ... -0.571493   
00:00:00.600000 -0.352664 -0.277353 -0.297844  0.188154  ... -0.723151   
00:00:00.800000 -0.371167 -0.430588 -0.320613  0.166308  ... -0.797590   
...                   ...       ...       ...       ...  ...       ...   
00:17:20.200000 -0.783829 -0.552474 -0.087489 -0.400377  ... -0.648197   
00:17:20.400000 -0.783829 -0.340877 -0.155995 -0.400377  ... -0.654596   
00:17:20.600000 -0.783829 -0.404122 -0.037461 -0.400377  ... -0.660809   
00:17:20.800000 -0.783829 -0.351787 -0.205840 -0.400377  ... -0.666843   
00:17:21        -0.783829 -0.590940 -0.372397 -0.400377  ... -0.672703   

                        9        26        28        29        30        37  \
00:00:00         0.156937 -0.696122  0.444113 -0.219053 -0.544912 -0.261650   
00:00:00.200000  0.105288 -0.351586  0.633657 -0.225043 -0.544912 -0.382058   
00:00:00.400000 -0.067953 -0.384992  0.489780 -0.230944 -0.544912 -0.482612   
00:00:00.600000 -0.197988 -0.481413  0.629067 -0.236759 -0.230613 -0.551955   
00:00:00.800000 -0.295591 -0.555278  0.823599 -0.242488 -0.273238 -0.595354   
...                   ...       ...       ...       ...       ...       ...   
00:17:20.200000 -0.721182 -0.356670  3.645533 -0.216210 -0.490532  7.505467   
00:17:20.400000 -0.707126 -0.371221  3.533746 -0.212437 -0.498738  7.332302   
00:17:20.600000 -0.117856 -0.400843  3.757240 -0.227802 -0.470555  7.420499   
00:17:20.800000 -0.456708 -0.413935  3.604610 -0.206952 -0.486733  7.431136   
00:17:21        -0.306329 -0.346471  4.047497 -0.218832 -0.409374  7.638934   

                       43        56     velocity_bins  
00:00:00        -0.260863 -0.399339     (2.829, 5.58]  
00:00:00.200000 -0.337916 -0.399339  (-0.0606, 2.829]  
00:00:00.400000 -0.460526 -0.399339  (-0.0606, 2.829]  
00:00:00.600000 -0.447324 -0.399339  (-0.0606, 2.829]  
00:00:00.800000 -0.367747 -0.399339  (-0.0606, 2.829]  
...                   ...       ...               ...  
00:17:20.200000 -0.799420  8.770218     (5.58, 8.332]  
00:17:20.400000 -0.799420  9.185146     (5.58, 8.332]  
00:17:20.600000 -0.799420  9.426400     (5.58, 8.332]  
00:17:20.800000 -0.799420  8.734445     (5.58, 8.332]  
00:17:21        -0.799420  9.467957     (5.58, 8.332]  

[5206 rows x 58 columns]

In [14]:
# clustering info per session 
coactivity_dfs = {}
for session in tqdm(list(grouped_raw_data.keys())):
    # get indicies of small cells 
    cells_to_drop = np.array([cell for cell in range(1, len(grouped_raw_data[session]['cell_contours'])+1) if 
                              len(np.array(np.where(grouped_raw_data[session]['for_dims'][cell]>0.6)[0]))<100])
    #compile and filter cell centers of mass
    filtered_centers_of_mass = grouped_raw_data[session]['com'].drop(cells_to_drop, axis =0)
    coactivity_dfs[session] = filtered_centers_of_mass

100%|██████████| 6/6 [00:02<00:00,  2.29it/s]


In [15]:
com_df = pd.concat(coactivity_dfs)

In [33]:
## save outputs if needed 
#com_df.loc['GRIN026_H16_M35_S34'].to_csv('/Volumes/My_Passport/cnmfe_analysis_files/OpenFieldAnalysis/2020/GRIN026_H16_M35_S34_com.csv')
#com_df.to_csv('/Volumes/My_Passport/cnmfe_analysis_files/OpenFieldAnalysis/2020/wt_com.csv')
save_path='/Volumes/My_Passport/cnmfe_analysis_files/OpenFieldAnalysis/2020/D1_mGluRKO_clustering/data/ko/'
for session in list(grouped_raw_data.keys()):
    com_df.loc[session].to_csv(save_path+session+'_com.csv')
    C_df.loc[session][C_df.loc[session]['velocity_bins'].notna()].iloc[:,:].dropna(axis=1).to_csv(save_path+session+'_filtered_f_traces.csv')
#com_df.to_csv('/projects/p30771/miniscope/analysis/OpenFieldAnalysis/spatial_clusters/jones_script_analysis/wt_com.csv')

In [17]:
list(grouped_raw_data.keys())

['GRIN009_H13_M59_S14',
 'GRIN018_H17_M41_S43',
 'GRIN018_H16_M13_S53',
 'GRIN011_H10_M19_S59',
 'GRIN012_H17_M32_S17',
 'GRIN012_H16_M57_S23']

In [22]:
C_df.loc['GRIN012_H17_M32_S17']

1         2         3         4         7         8  \
00:00:00        -0.376794 -0.004745  0.450959 -0.309227 -0.221392 -0.278460   
00:00:00.200000 -0.306167  0.102310  0.345854 -0.273399 -0.190540 -0.038487   
00:00:00.400000 -0.316236  0.000900  0.253256 -0.282653 -0.220512 -0.068930   
00:00:00.600000 -0.328850 -0.113600  1.501771 -0.316145 -0.261911 -0.177432   
00:00:00.800000 -0.340596 -0.208212  2.650304 -0.344636 -0.358440 -0.267445   
...                   ...       ...       ...       ...       ...       ...   
00:17:20.800000 -0.205112 -0.089171  1.040609 -0.496817 -0.751163 -0.721540   
00:17:21         0.220509 -0.274594  0.934513 -0.497935 -0.751163 -0.719434   
00:17:21.200000 -0.158256  0.095171  0.918597 -0.498929 -0.751163 -0.238494   
00:17:21.400000 -0.220229 -0.046662  1.098281 -0.499813 -0.751163 -0.605093   
00:17:21.600000  0.039137 -0.398520  0.741816 -0.500599 -0.751163  0.133077   

                       10        11        12        13  ...  253  254  255  \
00:00:00        -0.345085 -0.439040 -0.226133  0.154454  ...  NaN  NaN  NaN   
00:00:00.200000 -0.363913 -0.470494 -0.242395  0.046148  ...  NaN  NaN  NaN   
00:00:00.400000 -0.431485 -0.130533 -0.271917 -0.041017  ...  NaN  NaN  NaN   
00:00:00.600000 -0.352664 -0.277353 -0.297844  0.188154  ...  NaN  NaN  NaN   
00:00:00.800000 -0.371167 -0.430588 -0.320613  0.166308  ...  NaN  NaN  NaN   
...                   ...       ...       ...       ...  ...  ...  ...  ...   
00:17:20.800000 -0.783829 -0.351787 -0.205840 -0.400377  ...  NaN  NaN  NaN   
00:17:21        -0.783829 -0.590940 -0.372397 -0.400377  ...  NaN  NaN  NaN   
00:17:21.200000 -0.783829 -0.588025 -0.216089 -0.400377  ...  NaN  NaN  NaN   
00:17:21.400000 -0.783829 -0.499196 -0.208017 -0.400377  ...  NaN  NaN  NaN   
00:17:21.600000 -0.783829 -0.685502 -0.027561 -0.400377  ...  NaN  NaN  NaN   

                 257  259  260  263  264         56     velocity_bins  
00:00:00         NaN  NaN  NaN  NaN  NaN  -0.399339     (2.829, 5.58]  
00:00:00.200000  NaN  NaN  NaN  NaN  NaN  -0.399339  (-0.0606, 2.829]  
00:00:00.400000  NaN  NaN  NaN  NaN  NaN  -0.399339  (-0.0606, 2.829]  
00:00:00.600000  NaN  NaN  NaN  NaN  NaN  -0.399339  (-0.0606, 2.829]  
00:00:00.800000  NaN  NaN  NaN  NaN  NaN  -0.399339  (-0.0606, 2.829]  
...              ...  ...  ...  ...  ...        ...               ...  
00:17:20.800000  NaN  NaN  NaN  NaN  NaN   8.734445     (5.58, 8.332]  
00:17:21         NaN  NaN  NaN  NaN  NaN   9.467957     (5.58, 8.332]  
00:17:21.200000  NaN  NaN  NaN  NaN  NaN   9.160393               NaN  
00:17:21.400000  NaN  NaN  NaN  NaN  NaN  10.368723               NaN  
00:17:21.600000  NaN  NaN  NaN  NaN  NaN   8.250221               NaN  

[5209 rows x 242 columns]

In [16]:
## start analysis with Jones's scripts

In [50]:
C_df.loc['GRIN026_H16_M35_S34'].iloc[:,:].dropna(axis=1).values.transpose()[:-1,:]

array([[-0.25102836264285117, -0.17671887323743618, 0.08744908086959693,
        ..., 1.113494403225138, 1.1635455109482133, 0.9548691508208585],
       [0.2268380640848979, 0.23253915165516115, 0.18583093063192407,
        ..., 2.9236969370112105, 2.0648892423049814, 1.4124981472366254],
       [0.3454201962726108, 0.3291852956228375, 0.348359328978695, ...,
        1.7276403218368643, 0.9302150029215204, 0.40136977570200555],
       ...,
       [1.5080586594925516, 1.1376714598327757, 0.7888997955521182, ...,
        4.906288124011757, 4.563649419991397, 4.331158879753255],
       [-0.08612036548581312, 0.2692903540072777, 0.8126225505387242,
        ..., 1.431751306361636, 2.0773125413272573, 1.8421225231825937],
       [-0.5329814552150857, -0.5062164524852223, -0.6239475567078001,
        ..., -0.13858807216199112, -0.5112578693333647,
        -0.544580199721618]], dtype=object)

In [53]:
#get fluorescence traces for one animal
cell_traces = C_df.loc['GRIN026_H16_M35_S34'].iloc[:,:].dropna(axis=1).values.transpose()[:-1,:]

In [54]:
cell_traces

array([[-0.25102836264285117, -0.17671887323743618, 0.08744908086959693,
        ..., 1.113494403225138, 1.1635455109482133, 0.9548691508208585],
       [0.2268380640848979, 0.23253915165516115, 0.18583093063192407,
        ..., 2.9236969370112105, 2.0648892423049814, 1.4124981472366254],
       [0.3454201962726108, 0.3291852956228375, 0.348359328978695, ...,
        1.7276403218368643, 0.9302150029215204, 0.40136977570200555],
       ...,
       [1.5080586594925516, 1.1376714598327757, 0.7888997955521182, ...,
        4.906288124011757, 4.563649419991397, 4.331158879753255],
       [-0.08612036548581312, 0.2692903540072777, 0.8126225505387242,
        ..., 1.431751306361636, 2.0773125413272573, 1.8421225231825937],
       [-0.5329814552150857, -0.5062164524852223, -0.6239475567078001,
        ..., -0.13858807216199112, -0.5112578693333647,
        -0.544580199721618]], dtype=object)

In [55]:
np.shape(cell_traces)[0]

76

In [56]:
#xy centroids
cell_XY_coords = com_df.loc['GRIN026_H16_M35_S34'].iloc[:,:].dropna(axis=1).values.transpose()

In [57]:
#convert to MATLAB array 
cell_traces_MATLAB = matlab.double(cell_traces.tolist())
cell_XY_coords_MATLAB = matlab.double(cell_XY_coords.tolist())

In [58]:
#output signal peaks 
signal_peaks = eng.computeSignalPeaks(cell_traces_MATLAB, 'doMovAvg', 0, 'reportMidpoint', 1, 'numStdsForThresh', 2.5)

In [59]:
#pad signal peaks
padded_signal_peaks = eng.getPaddedSignalPeaks(signal_peaks)

In [60]:
#eng.size(signal_peaks)
eng.size(padded_signal_peaks)

matlab.double([[76.0,6250.0]])

In [61]:
#get cell distances multiply by pixel to micron scale
cellDistances = eng.pdist(cell_XY_coords_MATLAB, 'euclidean')*2.5

In [22]:
#get velocity bins
#reset timedelta index to integers matching frame numbers
vbin_frames_by_session = {}
for session in list(grouped_raw_data.keys()):
    df_w_integer_index = C_df.loc[session].reset_index()
    frame_indicies = {}
    for interval_range in all_sessions_v_bins.values.categories.values:
        frame_indicies[interval_range] = df_w_integer_index[df_w_integer_index['velocity_bins']==interval_range].index.values
    vbin_frames_by_session[session] = frame_indicies

/home/jma819/.conda/envs/caiman/lib/python3.6/site-packages/jupyter_client/jsonutil.py:67: DeprecationWarning: Interpreting naive datetime as local 2020-10-09 14:11:16.642508. Please add timezone info to timestamps.
  new_obj[k] = extract_dates(v)


In [23]:
#pull out indicies matching given bins 
#save list to csv file

for session in list(grouped_raw_data.keys()):
    for interval_range_int in range(len(all_sessions_v_bins.values.categories.values)):
        np.savetxt('/projects/p30771/miniscope/analysis/OpenFieldAnalysis/spatial_clusters/jones_script_analysis/'+str(session)+'_frame_bins_'+str(interval_range_int).zfill(3)+'_.csv',
                vbin_frames_by_session[session][list(frame_indicies.keys())[interval_range_int]])
    

/home/jma819/.conda/envs/caiman/lib/python3.6/site-packages/jupyter_client/jsonutil.py:67: DeprecationWarning: Interpreting naive datetime as local 2020-10-09 14:14:10.441681. Please add timezone info to timestamps.
  new_obj[k] = extract_dates(v)


In [102]:
#input frames to analyze
# all frames
frames_to_analyze = matlab.double(np.linspace(0, np.shape(cell_traces)[1]-1, np.shape(cell_traces)[1]).tolist())
within_frames_to_analyze = matlab.double(np.linspace(0, np.shape(cell_traces)[1]-1, np.shape(cell_traces)[1]).tolist())

In [103]:
# frames subset
v_bin_to_analyze = 0 

frames_to_analyze = matlab.double(frame_indicies[list(frame_indicies.keys())[v_bin_to_analyze]].tolist())
within_frames_to_analyze = matlab.double(frame_indicies[list(frame_indicies.keys())[v_bin_to_analyze]].tolist())

In [106]:
# cell distance bins
maxDist = matlab.double([500]) #compare out to 500 um
binSize = matlab.double([20]) #compare in 20 um bins
numBins = matlab.double([maxDist[0][0]/binSize[0][0]-1]) #throw out the first bin (overlapping cells)
binVector = matlab.double(np.linspace(int(binSize[0][0]), int(maxDist[0][0]), int(numBins[0][0])+1).tolist())

In [107]:
#other needed variables
numCells = matlab.double([np.shape(cell_traces)[0]])

In [114]:
# get Jaccard scores and compare to shuffle

eng.p355_jaccard_shuffle(padded_signal_peaks, frames_to_analyze[0], within_frames_to_analyze, cell_XY_coords_MATLAB, numCells, cellDistances, numBins, binVector, 1)


MatlabExecutionError: 
  File /Users/johnmarshall/Documents/Analysis/PythonAnalysisScripts/post_cmfe_analysis/openfield_analysis/spatial_clustering/jones_scripts/p355_jaccard_shuffle.m, line 29, in p355_jaccard_shuffle
Index in position 2 is invalid. Array indices must be positive integers or logical values.


In [ ]:
# clustering info per session 
coactivity_dfs = {}
for session in tqdm(list(grouped_raw_data.keys())):
    # get indicies of small cells 
    cells_to_drop = np.array([cell for cell in range(1, len(grouped_raw_data[session]['cell_contours'])+1) if 
                              len(np.array(np.where(grouped_raw_data[session]['for_dims'][cell]>0.6)[0]))<100])
    #compile and filter cell centers of mass
    filtered_centers_of_mass = grouped_raw_data[session]['com'].drop(cells_to_drop, axis =0)

    com_distances = utils_jjm.get_pairwise_distance_by_session(filtered_centers_of_mass)

    ##get binned fluorescence and calc Jaccard scores
    #arguments are sample widths to bin and z score threshold
    cells_in_session = C_df.loc[session].dropna(axis=1).drop('msCamFrame', axis=1)
    binned_fluorescence = cells_in_session.apply(utils_jjm.binning_function_uncrop, args=[1, 2])

    reindexed = binned_fluorescence.set_index(int(x) for x in np.linspace(0, len(binned_fluorescence)-1, len(binned_fluorescence)))

    #ks_results_2sided, ks_one_sided_more, ks_one_sided_less, coactivity_dfs = utils_jjm.spatial_coordination_by_session(reindexed, 10, com_distances)
    coactivity_dfs[session] = utils_jjm.spatial_coordination_by_session(reindexed)
    
    

In [ ]:
#save coactivity info to h5py file
for session in tqdm(list(grouped_raw_data.keys())):
    f = h5py.File("/Users/johnmarshall/Documents/Analysis/PythonAnalysisScripts/post_cmfe_analysis/openfield_analysis/spatial_clustering/" + str(session) +"clustering.h5", "w")

    for time_pt in range(len(coactivity_dfs[session])):
        dset = f.create_dataset(str(time_pt), np.shape(coactivity_dfs[session][time_pt].values), dtype=np.float32)
        dset[:, :] = coactivity_dfs[session][time_pt].values

    cell_pair_ids = f.create_dataset('cell_ids', np.shape(list(coactivity_dfs[session][0].columns)), dtype='i')
    cell_pair_ids[:, :] = list(coactivity_dfs[session][0].columns)

    f.close()

In [ ]:
spatial_coordination_by_session = {}
for session in list(grouped_raw_data.keys()):
    print(session)
    f = h5py.File("/Users/johnmarshall/Documents/Analysis/PythonAnalysisScripts/post_cmfe_analysis/openfield_analysis/spatial_clustering/" + str(session) +"clustering.h5", "r")
# get indicies of small cells 
    cells_to_drop = np.array([cell for cell in range(1, len(grouped_raw_data[session]['cell_contours'])+1) if 
                          len(np.array(np.where(grouped_raw_data[session]['for_dims'][cell]>0.6)[0]))<100])
#compile and filter cell centers of mass
    filtered_centers_of_mass = grouped_raw_data[session]['com'].drop(cells_to_drop, axis =0)
    com_distances = utils_jjm.get_pairwise_distance_by_session(filtered_centers_of_mass)

# get one sided ks tests from h5 data set
    binnums = 10
    ks_one_sided_more = []
    ks_one_sided_less = []
    for time_point in time_points:
        active_cells = [cell_ids[x] for x in [index[0] for index in np.argwhere(f[str(time_points[time_point])][0, 0:len(cell_ids)])]]
        active_cell_distances = [com_distances[cell].values[0] for cell in active_cells]
        cum_results_coactive = stats.cumfreq(active_cell_distances, numbins=binnums, defaultreallimits=(0, 500))
        less_result = stats.kstest(cum_results_coactive.cumcount/len(active_cell_distances), 'norm', alternative='less')
        more_result = stats.kstest(cum_results_coactive.cumcount/len(active_cell_distances), 'norm', alternative='greater')
        ks_one_sided_more.append(more_result)
        ks_one_sided_less.append(less_result)
    
    spatial_coordination_by_session[session] = {'ks_one_sided_more' : ks_one_sided_more, 'ks_one_sided_less' : ks_one_sided_less}

In [ ]:
session = list(grouped_raw_data.keys())[2]

In [ ]:
f = h5py.File("/Users/johnmarshall/Documents/Analysis/PythonAnalysisScripts/post_cmfe_analysis/openfield_analysis/spatial_clustering/" + str(session) +"clustering.h5", "r")
# get indicies of small cells 
cells_to_drop = np.array([cell for cell in range(1, len(grouped_raw_data[session]['cell_contours'])+1) if 
                          len(np.array(np.where(grouped_raw_data[session]['for_dims'][cell]>0.6)[0]))<100])
#compile and filter cell centers of mass
filtered_centers_of_mass = grouped_raw_data[session]['com'].drop(cells_to_drop, axis =0)
com_distances = utils_jjm.get_pairwise_distance_by_session(filtered_centers_of_mass)

In [ ]:
cell_ids = f['cell_ids']

In [ ]:
time_point = 95

In [ ]:
len(cell_ids)

In [ ]:
cell_ids[10]

In [ ]:
np.argwhere(f[str(time_point)][0, 0:len(cell_ids)])

In [ ]:
active_cells = [cell_idx[0] for cell_idx in np.argwhere(f[str(time_point)][0, 0:len(cell_ids)])]

In [ ]:
len(filtered_centers_of_mass)

In [ ]:
[cell_ids[cell] for cell in active_cells]

In [ ]:
cell_ids[active_cells[0]][0]

In [ ]:
cell_ids[active_cells[0]][1]

In [ ]:
com_distances[5]

In [ ]:
[com_distances[cell_ids[cell_idx][0]][cell_ids[cell_idx][1]].values[0] for cell_idx in active_cells]

In [ ]:
for time_pt in range(6251):
    if len(np.argwhere(f[str(time_pt)][0, 0:len(cell_ids)])) > 1:
        print(time_pt)
        break 
    

In [ ]:
len(f.keys())

In [ ]:
# get one sided ks tests from h5 data set
binnums = 10
ks_one_sided_more = []
ks_one_sided_less = []
cell_ids = f['cell_ids']
for time_point in time_points:
    print(time_point)
    active_cells = [cell_idx[0] for cell_idx in np.argwhere(f[str(time_point)][0, 0:len(cell_ids)])]
    active_cell_distances = [com_distances[cell_ids[cell_idx][0]][cell_ids[cell_idx][1]].values[0] for cell_idx in active_cells]
    cum_results_coactive = stats.cumfreq(active_cell_distances, numbins=binnums, defaultreallimits=(0, 500))
    less_result = stats.kstest(cum_results_coactive.cumcount/len(active_cell_distances), 'norm', alternative='less')
    more_result = stats.kstest(cum_results_coactive.cumcount/len(active_cell_distances), 'norm', alternative='greater')
    ks_one_sided_more.append(more_result)
    ks_one_sided_less.append(less_result)
    
spatial_coordination_by_session[session] = {'ks_one_sided_more' : ks_one_sided_more, 'ks_one_sided_less' : ks_one_sided_less}

In [ ]:
coord_index = np.nan_to_num([math.log(result.pvalue, 10)*-1 for result in ks_one_sided_less])

In [ ]:
plt.plot(coord_index[1700:2500])
plt.show()

In [ ]:
plt.plot(V_df.loc[session]['tail_base'].values[1700:2500])
plt.show()

In [ ]:
## for plotting average spatial coordination during trigger regions

In [ ]:

triggered_activity_across_sessions = {}
for session in list(grouped_raw_data.keys()):
    #inputs activity_threshold, resting_threshold, resting_baseline 
    crossing_indicies = utils_jjm.select_trigger_regions(binned_velocity_df[session], 0.5, 0.5, 20)
    #inputs time_to_plot
    threshold_activity = utils_jjm.average_triggered_regions(V_df.loc[session]['tail_base'].values, crossing_indicies, 40)
    
    f_threshold_activity = utils_jjm.average_triggered_regions(spatial_coordination_activity_in_session 
            , crossing_indicies, 40)
    triggered_activity_across_sessions[session] = pd.concat([threshold_activity, f_threshold_activity], axis=1, keys=['velocity', 'spatial_coordination'])

In [ ]:
#triggered_activity_across_sessions

In [ ]:
#concat by mouse 
concacted_by_mouse = {}
for mouse in list(set([session[0:7] for session in list(triggered_activity_across_sessions.keys())])):
    dfs_by_mouse = []
    sessions = []
    for session in list(triggered_activity_across_sessions.keys()):
        if mouse in session:
            if not(triggered_activity_across_sessions[session].empty):
                means = triggered_activity_across_sessions[session].mean(axis=1, level=0)
                dfs_by_mouse.append(means)
                sessions.append(session)
    if len(dfs_by_mouse)>0:
        concacted_by_mouse[mouse] = pd.concat(dfs_by_mouse, axis=1, keys=sessions)
combined_by_mouse = pd.concat(list(concacted_by_mouse.values()), axis=1, keys=list(concacted_by_mouse.keys()))

combined_by_mouse.head()

In [ ]:
binnums = 10
#ks_results_2sided = []
ks_one_sided_more = []
ks_one_sided_less = []
for time_point in tqdm(time_points):
    active_cells = [cell_ids[x] for x in [index[0] for index in np.argwhere(f[str(time_points[time_point])][0, 0:len(cell_ids)])]]
    coactive_cell_distances = [com_distances[cell].values[0] for cell in active_cells]
    #non_coactive_distances = com_distances[coactivity_df[coactivity_df == 0].dropna(axis=1).columns]
    #linear_distribution = np.linspace(1, len(coactive_cell_distances.columns), binnums)
    #binnums=10
    cum_results_coactive = stats.cumfreq(coactive_cell_distances, numbins=binnums, defaultreallimits=(0, 500))
    #cum_results_non_coactive = stats.cumfreq(non_coactive_distances.values[0], numbins=binnums, defaultreallimits=(0, 500))
    #plt.plot(np.linspace(0, 500, binnums), cum_results_coactive.cumcount/len(coactive_cell_distances.values[0]))
    #plt.plot(np.linspace(0, 500, binnums), cum_results_non_coactive.cumcount/len(non_coactive_distances.values[0]))
    #plt.plot(np.linspace(0, 500, binnums), linear_distribution/len(coactive_cell_distances.values[0]))
    #plt.show()
    less_result = stats.kstest(cum_results_coactive.cumcount/len(coactive_cell_distances), 'norm', alternative='less')
    more_result = stats.kstest(cum_results_coactive.cumcount/len(coactive_cell_distances), 'norm', alternative='greater')
    ks_one_sided_more.append(more_result)
    ks_one_sided_less.append(less_result)
    #coordination_index = utils_jjm.create_coordination_index(more_result, less_result)
    #two sided test between coactive and noncoactive distribution 
    #ks_result = stats.ks_2samp(cum_results_coactive.cumcount/len(coactive_cell_distances.values[0]), cum_results_non_coactive.cumcount/len(non_coactive_distances.values[0]))
    #coordination_indicies.append(coordination_index)
    #ks_results_2sided.append(ks_result)
    

In [ ]:
x = np.linspace(-40*.2, 40*.2, 80)
mean=combined_by_mouse.mean(axis=1, level=2)['velocity'].values
plt.plot(x, combined_by_mouse.mean(axis=1, level=2)['velocity'].values, color='k')
std_error = (combined_by_mouse.std(axis=1, level=2)['velocity'])/math.sqrt(combined_by_mouse.mean(axis=1, level=0).shape[1])
plt.fill_between(x, mean-std_error, mean+std_error)
ax = plt.gca()
ax.axvline(x=(0), linestyle='--', color='r')
plt.show()

In [ ]:
x = np.linspace(-40*.2, 40*.2, 80)
mean = combined_by_mouse.mean(axis=1, level=2)['fluorescence'].values
plt.plot(x, mean, color='k')
std_error = (combined_by_mouse.std(axis=1, level=2)['fluorescence'])/math.sqrt(combined_by_mouse.mean(axis=1, level=0).shape[1])
plt.fill_between(x, mean-std_error, mean+std_error)
ax = plt.gca()
ax.axvline(x=0, linestyle='--', color='r')
plt.show()